# Extract data from Wikipedia Infobox Templates into csv file
## Overview
* list_template_usage: takes Site object and Template name and returns a Generator
* extract_template_data: takes a Generator, a Template name, a Field name and returns a List

In [1]:
import pywikibot
import re

site = pywikibot.Site("en", 'wikipedia')

In [2]:
def template_name_replace_underscores(template_name):
    return template_name.replace('_', ' ')

In [3]:
from pywikibot import pagegenerators as pg

def list_template_usage(site_obj, template_name):
    """
    Takes Site object and template name and returns a generator.

    The function expects a Site object (pywikibot.Site()) and
    a template name (String). It creates a list of all
    pages using that template and returns them as a generator.
    The generator will load 50 pages at a time for iteration.
    """
    template_name = template_name_replace_underscores(template_name)
    name = "{}:{}".format(site_obj.namespace(10), template_name)
    tmpl_page = pywikibot.Page(site_obj, name)
    ref_gen = tmpl_page.getReferences(follow_redirects=False)
    filter_gen = pg.NamespaceFilterPageGenerator(ref_gen, namespaces=[0])
    generator = site_obj.preloadpages(filter_gen, pageprops=True)
    return generator

In [4]:
def convert_field(field):
    #convert template in feet
    match = re.search(r"{{(c|C)onvert\|\d+\.?\d*\|(ft|feet)(\||})", field)
    if match:
        ft = re.search(r"\d+\.?\d*", match[0])[0]
        return(ft,'ft')
    #convert template in metre
    match = re.search(r"{{(c|C)onvert\|\d+\.?\d*\|m(\||})", field)
    if match:
        metre = re.search(r"\d+\.?\d*", match[0])[0]
        return(metre,'m')
    #convert template in mm
    match = re.search(r"{{(c|C)onvert\|\d+\.?\d*\|mm(\||})", field)
    if match:
        mm = re.search(r"\d+\.?\d*", match[0])[0]
        return(mm,'mm')
    #convert template in kg
    match = re.search(r"{{(c|C)onvert\|\d+\.?\d*\|kg(\||})", field)
    if match:
        kg = re.search(r"\d+\.?\d*", match[0])[0]
        return(kg,'kg')
    #convert template in lb
    match = re.search(r"{{(c|C)onvert\|\d+\.?\d*\|lb(\||})", field)
    if match:
        lb = re.search(r"\d+\.?\d*", match[0])[0]
        return(lb,'lb')
    #convert template in sqft
    match = re.search(r"{{(c|C)onvert\|\d+\.?\d*\|sqft(\||})", field)
    if match:
        sqft = re.search(r"\d+\.?\d*", match[0])[0]
        return(sqft,'sqft')    
    #convert template in m2
    match = re.search(r"{{(c|C)onvert\|\d+\.?\d*\|m2(\||})", field)
    if match:
        m2 = re.search(r"\d+\.?\d*", match[0])[0]
        return(m2,'m2')  
    #field in metre
    match = re.search(r"\d+\.?\d*m|\d+\.?\d*\sm", field)
    if match:
        metre = re.search(r"\d+\.?\d*", match[0])[0]
        return(metre,'m')
    #field in ft
    match = re.search(r"\d+\.?\d*ft|\d+\.?\d*\sft", field)
    if match:
        ft = re.search(r"\d+\.?\d*", match[0])[0]
        return(ft,'ft')
    #field wikipedia entry -> wikidata qid
    match = re.search(r"(\[\[.*\||\[\[.*\]\])", field)
    if match:
        article = match[0].replace("[[","").replace("]]","")
        page = pywikibot.Page(site, article)
        qid = pywikibot.ItemPage.fromPage(page).id
        return("",qid)
    else:
        return(field,"")


print(convert_field("{{convert|9.15|m|abbr=on}}"))
print(convert_field("{{Convert|36|ft|m|abbr=on}}"))
print(convert_field("5.79m"))
print(convert_field("{{convert|4915|mm|abbr=on}}"))
print(convert_field("60ft"))
print(convert_field("LOA is several meters"))
print(convert_field("{{convert|58|ft}}"))
print(convert_field("{{convert|50|feet|abbr=on}}"))
print(convert_field("[[Bruce Farr]]"))
print(convert_field("[[Bruce Farr|BF]]"))
print(convert_field("{{convert|225|lb|kg|0|abbr=on}}"))
print(convert_field("{{convert|180|kg|lb|abbr=on}}"))


('9.15', 'm')
('36', 'ft')
('5.79', 'm')
('4915', 'mm')
('60', 'ft')
('LOA is several meters', '')
('58', 'ft')
('50', 'ft')
('', 'Q709233')
('', 'Q709233')
('225', 'lb')
('180', 'kg')


In [5]:
def extract_template_data(generator, template_name, field_name):
    """
    Takes a Generator, a Template name, a Field name and returns a List

    The function can also take a header (list of strings) that will be
    the headers of the table (Needs to be the same dimension as the table).
    The first column needs to be a link to property (It will be made into
    a link. In this example the second column is a list of links to Q-items.
    """
    template_name = template_name_replace_underscores(template_name)
    template_data = []
    header = ["qid", "url", field_name, "unit"]
    template_data.append(header)
 
    for page in generator:
        try:
            qid = pywikibot.ItemPage.fromPage(page).id
        except:
            print('no qid')
            qid = 'no qid'
        page_url = page.full_url()
        tmpl_list = page.raw_extracted_templates
        for tmpl in tmpl_list:
            if template_name in tmpl:
                try:
                    dico=tmpl[1]
                    (field,unit)=convert_field(dico[field_name])
                    print("qid: %s, url: %s | %s: %s %s" % (qid, page_url, field_name, field, unit))
                    template_data.append([qid, page_url, field, unit])
                except:
                    print('no such field')
                    
    return template_data

In [6]:
import csv

def list2csv(filename, list_name):
    """
    Takes a List and write a csv file
    """
    with open(filename, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerows(list_name)

In [7]:
template_name = "Infobox_sailboat_specifications"
field_name = "ballast"

#template_name = "Infobox sailing yacht"
#field_name = "length"

filename = template_name + "." + field_name + ".csv"
tmpl_gen = list_template_usage(site, template_name)
template_data = extract_template_data(tmpl_gen, template_name, field_name)

Retrieving 50 pages from wikipedia:en.


qid: Q442120, url: https://en.wikipedia.org/wiki/Knarr_%28keelboat%29 | ballast: 2822 lb
no such field
no such field
no such field
no such field
no such field
no such field
no such field
qid: Q5463514, url: https://en.wikipedia.org/wiki/Flying_Scot_%28dinghy%29 | ballast:  
qid: Q970466, url: https://en.wikipedia.org/wiki/Nordic_Folkboat | ballast: 2205 lb
qid: Q930962, url: https://en.wikipedia.org/wiki/Etchells | ballast: 2175 lb
no such field
no such field
no such field
no such field
no such field
qid: Q3183125, url: https://en.wikipedia.org/wiki/Jolie_Brise | ballast:  
qid: Q1684207, url: https://en.wikipedia.org/wiki/Javelin_dinghy | ballast: 49 lb
qid: Q594817, url: https://en.wikipedia.org/wiki/Yngling_%28keelboat%29 | ballast: 683 lb
no such field
qid: Q2277217, url: https://en.wikipedia.org/wiki/Shark_24 | ballast: 675 lb
qid: Q7786478, url: https://en.wikipedia.org/wiki/Thistle_%28dinghy%29 | ballast:  
qid: Q1752674, url: https://en.wikipedia.org/wiki/Albin_Vega | ballast: 

Retrieving 50 pages from wikipedia:en.


no such field
no such field
no such field
qid: Q1639639, url: https://en.wikipedia.org/wiki/SB20 | ballast: 327 kg
qid: Q4982397, url: https://en.wikipedia.org/wiki/Buccaneer_18 | ballast:  
no such field
qid: Q1418841, url: https://en.wikipedia.org/wiki/Fireball_%28dinghy%29 | ballast:  
no such field
qid: Q5151583, url: https://en.wikipedia.org/wiki/Comet_%28dinghy%29 | ballast:  
qid: Q7414553, url: https://en.wikipedia.org/wiki/San_Juan_24 | ballast: 1650 lb
no such field
qid: Q7018767, url: https://en.wikipedia.org/wiki/Newport_30 | ballast: 2500 lb
qid: Q7496198, url: https://en.wikipedia.org/wiki/Shields_%28keelboat%29 | ballast: 3080 lb
qid: Q6188785, url: https://en.wikipedia.org/wiki/Jet_14 | ballast:  
no such field
no such field
qid: Q1253532, url: https://en.wikipedia.org/wiki/Formula_18 | ballast:  
no such field
qid: Q7440009, url: https://en.wikipedia.org/wiki/Sea_Sprite_34 | ballast: 5000 lb
qid: Q14524820, url: https://en.wikipedia.org/wiki/IMOCA_60 | ballast:  
qid: 

Retrieving 50 pages from wikipedia:en.


...

qid: Q104145305, url: https://en.wikipedia.org/wiki/Scampi_30 | ballast: 2778 lb
qid: Q104144429, url: https://en.wikipedia.org/wiki/Alerion_Express_19 | ballast: 300 lb
qid: Q104880418, url: https://en.wikipedia.org/wiki/Singoalla_34 | ballast: 3750 lb
qid: Q104188765, url: https://en.wikipedia.org/wiki/Viggen_23 | ballast: 1323 lb
qid: Q10667096, url: https://en.wikipedia.org/wiki/Shipman_28 | ballast: 2667 lb
qid: Q104880824, url: https://en.wikipedia.org/wiki/Albin_7.8 | ballast: 2094 lb
qid: Q104880832, url: https://en.wikipedia.org/wiki/Alpha_29 | ballast: 3042 lb
qid: Q104880847, url: https://en.wikipedia.org/wiki/Delta_31 | ballast: 3505 lb
qid: Q104881060, url: https://en.wikipedia.org/wiki/Allegra_24 | ballast: 2200 lb
qid: Q104881584, url: https://en.wikipedia.org/wiki/Greenwich_24 | ballast: 1500 lb
qid: Q104881672, url: https://en.wikipedia.org/wiki/Alegria_67 | ballast: none 
qid: Q104881845, url: https://en.wikipedia.org/wiki/AMF_2100 | ballast: 850 lb
qid: Q104290513, u

Retrieving 50 pages from wikipedia:en.


qid: Q105360264, url: https://en.wikipedia.org/wiki/Cal_21 | ballast: 360 lb
qid: Q105393819, url: https://en.wikipedia.org/wiki/Cal_22 | ballast: 775 lb
qid: Q105082255, url: https://en.wikipedia.org/wiki/Cal_24 | ballast: 1000 lb
qid: Q105438874, url: https://en.wikipedia.org/wiki/Cal_2-24 | ballast: 1400 lb
qid: Q105453339, url: https://en.wikipedia.org/wiki/Cal_3-24 | ballast: 1175 lb
qid: Q105474389, url: https://en.wikipedia.org/wiki/Cal_T%2F4 | ballast: 2000 lb
qid: Q105561953, url: https://en.wikipedia.org/wiki/Com-Pac_Legacy | ballast: 400 lb
qid: Q105580848, url: https://en.wikipedia.org/wiki/Marlin_23 | ballast: 1400 lb
qid: Q105333135, url: https://en.wikipedia.org/wiki/Cape_Dory_25 | ballast: 1700 lb
qid: Q105356576, url: https://en.wikipedia.org/wiki/Cape_Dory_25D | ballast: 2050 lb
qid: Q105626394, url: https://en.wikipedia.org/wiki/Typhoon_Senior | ballast: 1700 lb
qid: Q105638481, url: https://en.wikipedia.org/wiki/Capri_16 | ballast: 425 lb
qid: Q104233882, url: https

Retrieving 37 pages from wikipedia:en.


qid: Q106436039, url: https://en.wikipedia.org/wiki/ETAP_28i | ballast: 1808 lb
qid: Q106436037, url: https://en.wikipedia.org/wiki/ETAP_30 | ballast: 3032 lb
qid: Q106436046, url: https://en.wikipedia.org/wiki/ETAP_30i | ballast: 2425 lb
qid: Q106436042, url: https://en.wikipedia.org/wiki/ETAP_32i | ballast: 2866 lb
qid: Q106436048, url: https://en.wikipedia.org/wiki/ETAP_39s | ballast: 4960 lb
qid: Q106436031, url: https://en.wikipedia.org/wiki/ETAP_22 | ballast: 1213 lb
qid: Q106522259, url: https://en.wikipedia.org/wiki/ETAP_28 | ballast: 2557 lb
qid: Q106436035, url: https://en.wikipedia.org/wiki/ETAP_23i | ballast: 1036 lb
qid: Q106436036, url: https://en.wikipedia.org/wiki/ETAP_26 | ballast: 1568 lb
qid: Q106436041, url: https://en.wikipedia.org/wiki/ETAP_38i | ballast: 4850 lb
qid: Q106436043, url: https://en.wikipedia.org/wiki/ETAP_35i | ballast: 3836 lb
qid: Q106436047, url: https://en.wikipedia.org/wiki/ETAP_21i | ballast: 772 lb
qid: Q106436045, url: https://en.wikipedia.or

In [8]:
list2csv(filename, template_data)

# Push to Wikidata

In [9]:
#https://www.wikidata.org/wiki/Wikidata:Pywikibot_-_Python_3_Tutorial/Setting_statements
#Setting statements
# -*- coding: utf-8  -*-
import pywikibot
"""
First check if P2373 already exists for Q15935. If not add claim P2373 with the string 'Kanye_west'.
This works for claims that only allow a single value.
"""
site_data = pywikibot.Site("wikidata", "wikidata")
repo_data = site_data.data_repository()
item_data = pywikibot.ItemPage(repo_data, "Q97179551")
claims = item_data.get('claims') #Get all the existing claims
item_data.get() #Fetch all page data, and cache it.

In [10]:
claims['claims']

<class 'pywikibot.page._collections.ClaimCollection'>({'P2671': [Claim.fromJSON(DataSite("wikidata", "wikidata"), {'mainsnak': {'snaktype': 'value', 'property': 'P2671', 'datatype': 'external-id', 'datavalue': {'value': '/g/11jbgj2p09', 'type': 'string'}}, 'type': 'statement', 'id': 'Q97179551$79DCF988-9F42-447C-9D9D-2F95BEB298AF', 'rank': 'normal'})], 'P31': [Claim.fromJSON(DataSite("wikidata", "wikidata"), {'mainsnak': {'snaktype': 'value', 'property': 'P31', 'datatype': 'wikibase-item', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 106179098}, 'type': 'wikibase-entityid'}}, 'type': 'statement', 'id': 'Q97179551$11de689a-4d8e-9ea0-ff6b-2dc203fe3e21', 'rank': 'normal', 'references': [{'snaks': {'P854': [{'snaktype': 'value', 'property': 'P854', 'datatype': 'url', 'datavalue': {'value': 'https://jboats.com/j92s-tech-specs', 'type': 'string'}}]}, 'snaks-order': ['P854'], 'hash': '4a60fed570bf741fc8e5ea00015e6c537aab0773'}]})], 'P2043': [Claim.fromJSON(DataSite("wikidata",

In [11]:
statement_property = 'P2067'
statement_value = 2822
statement_unit = 'lb'


if statement_property in claims[u'claims']: #if a value is present already, print error.
    pywikibot.output(u'Error: statement already exists!')
else:
    stringclaim = pywikibot.Claim(repo_data, statement_property) #Else, add the value
    stringclaim.setTarget(statement_value)
    item_data.addClaim(stringclaim, summary=u'adding statement via bot')

Error: statement already exists!


In [14]:
claims['P2067']

KeyError: 'P2067'

In [78]:
# -*- coding: utf-8  -*-
import pywikibot
"""
Adding a qualifier to existing claims/statements
"""
site_data = pywikibot.Site("wikidata", "wikidata")
repo_data = site_data.data_repository()
item_data = pywikibot.ItemPage(repo_data, "Q97179551")
statement_property = 'P2067' # 'P2067' mass
statement_qualifier = 'P642' # 'P642' of
statement_qualifier_target = 'Q5636358' # 'Q5636358' displacement

statement_value = 2822
statement_unit = 'lb'


item_data.get() #Fetch all page data, and cache it.



for claim in item_data.claims[statement_property]: #Finds all statements with property (P)
    if statement_qualifier not in claim.qualifiers: #If not already exist
        print('statement_qualifier not found')
    else:
        for values in claim.qualifiers[statement_qualifier]:
            if statement_qualifier_target == values.target.id:
                print('statement_qualifier_target already defined')
            else:
                print('statement_qualifier_target not found')

        # Generate QUALIFIER FOR EXISTENCE STATEMENTS/CLAIMS
        #qualifier = pywikibot.Claim(repo, u'P642')
        #target = pywikibot.ItemPage(repo, "Q35409")
        #qualifier.setTarget(target)

        #claim.addQualifier(qualifier, summary=u'Adding a qualifier.') #Adding qualifier to all statements (P131)

AttributeError: 'list' object has no attribute 'target'

In [80]:
item_data.claims['P2067'][0].qualifiers['P642'][0].target

ItemPage('Q5636358')

In [89]:
import pprint

def prettyPrint(variable):
    pp = pprint.PrettyPrinter(indent=4)
    pp.pprint(variable)

In [90]:
prettyPrint(item_data.claims['P2067'])

[   Claim.fromJSON(DataSite("wikidata", "wikidata"), {'mainsnak': {'snaktype': 'value', 'property': 'P2067', 'datatype': 'quantity', 'datavalue': {'value': {'amount': '+2550', 'upperBound': None, 'lowerBound': None, 'unit': 'http://www.wikidata.org/entity/Q11570'}, 'type': 'quantity'}}, 'type': 'statement', 'id': 'Q97179551$2cb0f681-4bcd-ede2-a147-7aa352b7ca64', 'rank': 'normal', 'qualifiers': {'P642': [{'snaktype': 'value', 'property': 'P642', 'datatype': 'wikibase-item', 'datavalue': {'value': {'entity-type': 'item', 'numeric-id': 5636358}, 'type': 'wikibase-entityid'}, 'hash': '9b35e18a0b409cbb243dcafd61faca309a7677bc'}]}, 'qualifiers-order': ['P642'], 'references': [{'snaks': {'P854': [{'snaktype': 'value', 'property': 'P854', 'datatype': 'url', 'datavalue': {'value': 'https://jboats.com/j92s-tech-specs', 'type': 'string'}}], 'P813': [{'snaktype': 'value', 'property': 'P813', 'datatype': 'time', 'datavalue': {'value': {'time': '+00000002021-03-30T00:00:00Z', 'precision': 11, 'after

# Alternative

In [ ]:
def strip_dict_entries(dict_in):
    """Strip whitespace from all keys and (string) values in a dictionary."""
    dict_out = dict()
    if not isinstance(dict_in, dict):
        raise MyError('strip_dict_entries() expects a dictionary object'
                      'as input but found "%s"' % type(dict_in).__name__)
    for k, v in dict_in.items():
        if isinstance(v,str):
            v = v.strip()
        dict_out[k.strip()] = v
    return dict_out

In [ ]:
def extract_template_data2(generator, template_name, field_name, header=None):
    """
    Takes a Generator, a Template name, a Field name and returns a List

    The function can also take a header (list of strings) that will be
    the headers of the table (Needs to be the same dimension as the table).
    The first column needs to be a link to property (It will be made into
    a link. In this example the second column is a list of links to Q-items.
    """
    template_data = []
    result = []

    if header != None:
        template_data.append(header)

    for page in generator:                    
        templates = pywikibot.textlib.extract_templates_and_params(page.text)
        for tp in templates:
            if tp[0] == template_name:
                #print(tp[1])
                result.append(strip_dict_entries(tp[1]))
            
    return result

In [ ]:
result

In [ ]:
def get_all_template_entries(wikitext, template_name):
    """Return a list of all arguments for instances of a given template."""
    templates = pywikibot.textlib.extract_templates_and_params(wikitext)
    result = []
    for tp in templates:
        if tp[0] == template_name:
            print(tp[1])
            result.append(strip_dict_entries(tp[1]))
    return result

In [ ]:
toto = get_all_template_entries(page.text, template_name)
toto

In [ ]:
import mwparserfromhell
import pywikibot

def parse(title):
    site = pywikibot.Site()
    page = pywikibot.Page(site, title)
    text = page.get()
    return mwparserfromhell.parse(text)



In [ ]:
for page in tmpl_gen:
    item = pywikibot.ItemPage.fromPage(page)
    print(page.title(), item.getID())
    print(page)

In [ ]:
type(page)

In [ ]:
templates = page.templatesWithParams()
(template_page,item)=templates[0]
print(type(item))
item

In [ ]:
templates_raw = page.raw_extracted_templates
(template_0, item_0)=templates_raw[0]
print(type(item_0))
item_0

In [ ]:
item_0["loa"]

In [ ]:
for (template, fielddict) in templates:
            # Clean up template
            try:
                template = pywikibot.Page(page.site, template,
                                          ns=10).title(with_ns=False)
            except pywikibot.exceptions.InvalidTitle:
                pywikibot.error(
                    "Failed parsing template; '{}' should be "
                    'the template name.'.format(template))
                continue
            # We found the template we were looking for
            for field, value in fielddict.items():
                field = field.strip()

In [ ]:
for (template, fielddict) in templates:
    print(template)
    print(fielddict)

In [ ]:
fielddict

In [ ]:
(template, fielddict)=templates[0]

In [ ]:
fielddict

In [ ]:
(field, value) = fielddict.items(2)

In [ ]:
import pywikibot
from pywikibot import pagegenerators as pg

def list_template_usage(site_obj, template_name):
    """
    Takes Site object and template name and returns a generator.

    The function expects a Site object (pywikibot.Site()) and
    a template name (String). It creates a list of all
    pages using that template and returns them as a generator.
    It only returns pages in the 121-namespace (property-talk-pages).
    The generator will load 50 pages at a time for iteration.
    """
    name = "{}:{}".format(site.namespace(10), template_name)
    tmpl_page = pywikibot.Page(site, name)
    ref_gen = pg.ReferringPageGenerator(tmpl_page, onlyTemplateInclusion=True)
    filter_gen = pg.NamespaceFilterPageGenerator(ref_gen, namespaces=[10])
    generator = site.preloadpages(filter_gen, pageprops=True)
    return generator

In [ ]:
site = pywikibot.Site("en", 'wikipedia')
tmpl_gen = list_template_usage(site, "Infobox_sailboat_specifications")

# code for single page

In [21]:
import pywikibot

#template_name = "Infobox_sailboat_specifications"
#field_name = "loa"
template_name = "Infobox sailing yacht"
field_name = "designer"
filename = template_name + "." + field_name + ".csv"

site = pywikibot.Site("en", "wikipedia")
#page = pywikibot.Page(site, "ETAP_32s")
#page = pywikibot.Page(site, "Finn (dinghy)")
#page = pywikibot.Page(site, "49er (dinghy)")
#page = pywikibot.Page(site, "Squib (keelboat)")
#page = pywikibot.Page(site, "Hobie 16")
page = pywikibot.Page(site, "Maiden_%28yacht%29")
page_url = page.full_url()
qid = pywikibot.ItemPage.fromPage(page).id
print(qid)

Q60755728


In [22]:
tmpl_list = page.raw_extracted_templates
template_name = template_name_replace_underscores(template_name)
for tmpl in tmpl_list:
    if template_name in tmpl:
        try:
            dico=tmpl[1]
            field=dico[field_name]
            print("qid: %s, page: %s | %s: %s" % (qid, page_url, field_name, field))
            template_data.append([qid, page_url, field])
        except:
            print('no such field')

qid: Q60755728, page: https://en.wikipedia.org/wiki/Maiden_%28yacht%29 | designer: [[Bruce Farr]]


In [16]:
print(convert_field("{{convert|58|ft}}"))

('{{convert|58|ft}}', '')


In [23]:
field

'[[Bruce Farr]]'

In [24]:
type(field)

str

In [28]:
page = pywikibot.Page(site, field)
item = pywikibot.ItemPage.fromPage(page)

print(item)

InvalidTitle: '[[Bruce Farr]]' contains illegal char(s) '['